In [2]:
include("TN_Superfermionic_Mesoscopic_Leads/Code/Interacting_functions.jl")

Energy_Current_HS_time_dependent (generic function with 2 methods)

Let's consider the same Spectral density and logarithmic linear discretization that we have been using the whole time:

In [13]:
using LinearAlgebra
using FlexiMaps

W = 8.0
Γ = W/8
J(ω) = abs(ω) <= abs(W) ? Γ : 0 #Spectral density Γ = W/8 in this case

function Logarithmic_linear_arrays(L, J)
    #L: lead size
    #J: Spectral density    
    
    #We need to define εk for the lead. In particular, we use the logarithmic-linear discretization of the paper. As this is just for the article, it is not worthy to put it with the other functions in NonInteacting_functions.jl
    
    W_log = W #W
    W_lin = W_log/2 #W* = W/2
    
    L_log = L*0.2
    L_lin = L - L_log
    
    Lin_Window = LinRange(-W_lin, W_lin, Int(L_lin))
    Log_Window = maprange(log, W_lin, W_log, length=Int(L_log/2 + 1))
    
    εk = Float64[]
    append!(εk, -reverse( Log_Window[2:length(Log_Window)]), Lin_Window, Log_Window[2:length(Log_Window)])
    
    γk = [εk[k+1] - εk[k] for k=1:Int(L/2)]
    append!(γk, reverse(γk))

    κkp = ComplexF64[sqrt(J(εk[k])*γk[k]/(2.0*pi)) for k=1:L] #Kp = sqrt(J(e)*γ/(2*pi))

    return εk, γk,  κkp
end

Logarithmic_linear_arrays (generic function with 1 method)

In [14]:
#Global variables by default
μ_L, μ_R = W/16, W/16 #As we want to focus in Heat rectification, we should not have gradient of chemical potential.

ts = (W/8)

function fk_arrays(εk_array_L, εk_array_R)
    
    fk_array_L = [fermi_dirac_distribution(ε, μ_L, β_L) for ε = εk_array_L]
    fk_array_R = [fermi_dirac_distribution(ε, μ_R, β_R) for ε = εk_array_R]
    
    return fk_array_L, fk_array_R
end

fk_arrays (generic function with 1 method)

In [15]:
β_L, β_R = 1/(10*ts), 1/(2*ts) #Forward Bias: TL > TR
# β_L, β_R = 1/(2*ts), 1/(10*ts) #Reverse Bias: TL < TR

(0.1, 0.5)

Possible sizes: L = 10, 20. D = 2, 4

In [35]:
#Lead Parameters
L, D = 10, 10
εk, γk, κp = Logarithmic_linear_arrays(L, J)
fk_L, fk_R = fk_arrays(εk, εk);

In [36]:
#System Parameters
D = 2
U = 1.2*ts
E = 4
μ = -E*(D+1)/4

ε_system = [(μ+0.5E*j) for j =1:D]; #Tilted system energies

In [40]:
M = L + D + R

sites = siteinds("S=1/2",2*M)
I_vec = Build_left_vacuum(sites);

#TN Parameters
maxdim = 40;
dt = 0.05

0.05

In [51]:
#Let's use the thermal state as the start point for the time evolution:

Swap_Gates, TEBD_Gates = Build_Gates(sites, εk, γk, 0*κp, fk_L, εk, γk, 0*κp, fk_R, ε_system, ts, U); #0*Kp_array = no coupling between the lead and the system

NumSteps = 1000
@time Thermal_State, Occupation_Matrix_t = Apply_TEBD(I_vec, I_vec, Swap_Gates, TEBD_Gates, NumSteps); #maxdim = 40 by default
@show JP = Particle_Current(Thermal_State, I_vec, γk, fk_L, Float64[], Float64[])
@show JE = Energy_Current(Thermal_State, I_vec, γk, fk_L, εk, κp, Float64[], Float64[], Float64[], ComplexF64[])

10.0 % Completed.
20.0 % Completed.
30.0 % Completed.
40.0 % Completed.
50.0 % Completed.
60.0 % Completed.
70.0 % Completed.
80.0 % Completed.
90.0 % Completed.
100.0 % Completed.
 48.451494 seconds (340.25 M allocations: 92.832 GiB, 8.15% gc time)


In [53]:
Thermal_State

MPS
[1] ((dim=2|id=606|"S=1/2,Site,n=1"), (dim=2|id=348|"Link,n=1"))
[2] ((dim=2|id=348|"Link,n=1"), (dim=2|id=798|"S=1/2,Site,n=2"), (dim=1|id=128|"Link,n=2"))
[3] ((dim=2|id=897|"S=1/2,Site,n=3"), (dim=1|id=128|"Link,n=2"), (dim=2|id=139|"Link,n=1"))
[4] ((dim=2|id=139|"Link,n=1"), (dim=2|id=871|"S=1/2,Site,n=4"), (dim=1|id=619|"Link,n=2"))
[5] ((dim=2|id=830|"S=1/2,Site,n=5"), (dim=1|id=619|"Link,n=2"), (dim=2|id=103|"Link,n=1"))
[6] ((dim=2|id=103|"Link,n=1"), (dim=2|id=858|"S=1/2,Site,n=6"), (dim=1|id=760|"Link,n=2"))
[7] ((dim=2|id=953|"S=1/2,Site,n=7"), (dim=1|id=760|"Link,n=2"), (dim=2|id=361|"Link,n=1"))
[8] ((dim=2|id=361|"Link,n=1"), (dim=2|id=226|"S=1/2,Site,n=8"), (dim=1|id=616|"Link,n=2"))
[9] ((dim=2|id=297|"S=1/2,Site,n=9"), (dim=1|id=616|"Link,n=2"), (dim=2|id=417|"Link,n=1"))
[10] ((dim=2|id=417|"Link,n=1"), (dim=2|id=728|"S=1/2,Site,n=10"), (dim=1|id=406|"Link,n=2"))
[11] ((dim=2|id=908|"S=1/2,Site,n=11"), (dim=1|id=406|"Link,n=2"), (dim=2|id=139|"Link,n=1"))
[12] ((